In [92]:
import pdfplumber
import pandas as pd
import os
import regex as re
from collections import OrderedDict

In [118]:
infile = "/mnt/raidtmp/joppich/pubmed_pmc/pmc/vimtm/covidsents/PMC8307234/PMC8307234_cells-10-01762-s001/cells-1268410-supplementary.pdf"
#infile = "/mnt/raidtmp/joppich/pubmed_pmc/pmc/vimtm/covid_docs/Liu_2021/liu_supplements.pdf"

In [114]:
fastaSequenceMirRE = re.compile(r'^[ATCGU]{15,30}$')
mirnaFasta = re.compile(r'>.*\n[ATCGU]{15,30}$')
fastaStartRE = re.compile(r'^>\S\S\S\S+')

def is_empty(elem):
    if elem is None:
        return True

    if len(str(elem)) == 0:
        return True

    return False


def test_is_sequence( word):
    mirSequence = str(word).replace("Sequence: ", "").strip()
    mirSequence = re.sub(r"^[\s0-9]+","",word)

    match = fastaSequenceMirRE.search(mirSequence)

    if match:
        return True

    match = mirnaFasta.search(mirSequence)
    if match:
        return True

    return False
    

def column_contains_mirnas(df, indexRow, xi, num_tests=10):

    foundSeqs = 0
    testedSeqs = 0
    for ri, row in df.loc[indexRow+1:min(num_tests, df.shape[0]),].iterrows():
        
        word = str(row[xi])

        #print(word, test_is_sequence(word))
        if not is_empty(word):
            testedSeqs+= 1

        if test_is_sequence(word):
            foundSeqs += 1
        #else:
        #    print("F", word)

    if testedSeqs>0 and foundSeqs/testedSeqs > 0.8:
        return True

    print("[column_contains_mirnas] Column", xi, "is not a valid miRNA Sequence", foundSeqs, testedSeqs)
    return False

def hasSeqColumn(df, num_tests=10):
    hasSeqColumns = False
    for i in range(pageDF.shape[1]):
        isSeqCol = column_contains_mirnas(pageDF, 0, i, num_tests=num_tests)
        print("col", i, isSeqCol)
        hasSeqColumns = hasSeqColumns or isSeqCol

    return hasSeqColumns

In [119]:
pdf = pdfplumber.open(infile)

extractionSettings = OrderedDict()
extractionSettings["loose"] = {
            "vertical_strategy": "text", 
            "horizontal_strategy": "lines",
            #"snap_tolerance": 3,
            "snap_x_tolerance": 5,
            "snap_y_tolerance": 3,
            "join_x_tolerance": 5,
            "join_y_tolerance": 3,
            "edge_min_length": 3,
            "min_words_vertical": 3,
            "min_words_horizontal": 5,
            "keep_blank_chars": False,
            "text_x_tolerance": 5,
            "text_y_tolerance": 3,
            "intersection_x_tolerance": 5,
            "intersection_y_tolerance": 3,
            }
extractionSettings["tight"] = {
        "vertical_strategy": "text", 
        "horizontal_strategy": "text",
        "snap_x_tolerance": 1,
        "snap_y_tolerance": 5,
        "join_x_tolerance": 1,
        "join_y_tolerance": 3,
        "edge_min_length": 3,
        "min_words_vertical": 3,
        "min_words_horizontal": 1,
        "keep_blank_chars": False,
        "text_x_tolerance": 1,
        "text_y_tolerance": 3,
        "intersection_x_tolerance": 1,
        "intersection_y_tolerance": 3,
    }


pdfDFs = {}

for eSetting in extractionSettings:
    print(eSetting)
    for page in pdf.pages:
       
        curRow = None
        tableRows = []

        for table in page.extract_tables(extractionSettings[eSetting]):
            for row in table:
                #print("R", row)
                
                if row[0] != None:

                    if curRow != None:
                        curRow = [x.replace("\n", "") if not x is None else "" for x in curRow]
                        tableRows.append(curRow)

                    curRow = row
                    continue

                if not curRow is None:
                    for xi in range(0, len(curRow)):
                        if row != None and row[xi] != None:

                            if curRow[xi] is None:
                                curRow[xi] = row[xi]
                            else:
                                curRow[xi] += "\n" + row[xi]
                

        pageDF = pd.DataFrame(tableRows)

        if not 0 in pageDF.shape:

            
            if page.page_number in pdfDFs:

                print(page.page_number)

                pageHasSeqCol = hasSeqColumn(pageDF, num_tests=20)
                existHasSeqCol = hasSeqColumn(pdfDFs[page.page_number], num_tests=20)

                if pageHasSeqCol and not existHasSeqCol:
                    #print("Replacing PageDF", page.page_number)
                    pass
                elif pageHasSeqCol and pdfDFs[page.page_number].shape[0] * pdfDFs[page.page_number].shape[1] < pageDF.shape[0]*pageDF.shape[1]:
                    #print("Replacing PageDF", page.page_number)
                    pass
                else:
                    continue

            pdfDFs[page.page_number] = pageDF


loose
2 (2, 7)
5 (2, 10)
8 (31, 4)
9 (15, 7)
10 (16, 7)
11 (15, 7)
12 (38, 8)
13 (41, 7)
14 (40, 7)
15 (80, 3)
16 (40, 7)
17 (40, 6)
18 (31, 9)
19 (16, 4)
20 (3, 4)
tight
1 (63, 9)
2
[column_contains_mirnas] Column 0 is not a valid miRNA Sequence 0 0
col 0 False
[column_contains_mirnas] Column 1 is not a valid miRNA Sequence 0 10
col 1 False
[column_contains_mirnas] Column 2 is not a valid miRNA Sequence 0 0
col 2 False
[column_contains_mirnas] Column 3 is not a valid miRNA Sequence 0 6
col 3 False
[column_contains_mirnas] Column 4 is not a valid miRNA Sequence 0 3
col 4 False
[column_contains_mirnas] Column 5 is not a valid miRNA Sequence 0 3
col 5 False
[column_contains_mirnas] Column 6 is not a valid miRNA Sequence 0 2
col 6 False
[column_contains_mirnas] Column 7 is not a valid miRNA Sequence 0 2
col 7 False
[column_contains_mirnas] Column 8 is not a valid miRNA Sequence 0 2
col 8 False
[column_contains_mirnas] Column 9 is not a valid miRNA Sequence 0 2
col 9 False
[column_contains

In [120]:
pdfDFs[9]

,0,1,2,3,4,5,6
0,pre-miRNA ID,Start,End,MFE(kcal/mol),Sequence of pre-miRNA,5p miRNA,3p miRNA
1,MD113,7961,8026,–26.8,UGUCAACCUAUACUGUUACUAGAUCAGGCAUUAGUGUCUGAUGUUG...,ACCUAUACUGUUACUAGAUCAG,GAUGUUGGUGAUAGUGCGGAAG
2,MD129,8787,8960,–55.64,GUGGUGGUAGUUAUACUAAUGACAAAGCUUGCCCAUUGAUUGCUGC...,UAUACUAAUGACAAAGCUUGCC,CACGAUAUUACGCACAACUAAU
3,MD172,11234,11334,–36.6,GCUAGUUGGGUGAUGCGUAUUAUGACAUGGUUGGAUAUGGUUGAUA...,AUACUAGUUUGUCUGGUUUUAA,UGUGUUAUGUAUGCAUCAGCUG
4,MD198,13171,13263,–32.7,UGGUACUGGUCAGGCAAUAACAGUUACACCGGAAGCCAAUAUGGAU...,UAUGGAUCAAGAAUCCUUUGGU,UUUGGUGGUGCAUCGUGUUGUC
5,MD2,347,398,–22.7,GUACGUGGCUUUGGAGACUCCGUGGAGGAGGUCUUAUCAGAGGCAC...,UUGGAGACUCCGUGGAGGAGGU,AGGUCUUAUCAGAGGCACGUCA
6,MD202,13379,13436,–20.5,GUCUGCGGUAUGUGGAAAGGUUAUGGCUGUAGUUGUGAUCAACUCC...,AAAGGUUAUGGCUGUAGUUGUG,UGAUCAACUCCGCGAACCCAUG
7,MD21,2033,2105,–24.01,UUGGCUACUAACAAUCUAGUUGUAAUGGCCUACAUUACAGGUGGUG...,UAAUGGCCUACAUUACAGGUGG,UUACAGGUGGUGUUGUUCAGUU
8,MD250,17048,17120,–29.6,AGGUUGGUAUGCAAAAGUAUUCUACACUCCAGGGACCACCUGGUAC...,UAUGCAAAAGUAUUCUACACUC,UGGUAAGAGUCAUUUUGCUAUU
9,MD251,17134,17197,–30.6,CCUUCUGCUCGCAUAGUGUAUACAGCUUGCUCUCAUGCCGCUGUUG...,UGCUCGCAUAGUGUAUACAGCU,UGUUGAUGCACUAUGUGAGAAG
